In [ ]:
import os
import numpy as np
from tqdm import tqdm
from glob import glob
from numpy import zeros
from numpy.random import randint
import torch
import os
import cv2
from statistics import mean
from torch.nn.functional import threshold, normalize

# Data Viz
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2


In [ ]:
! pip install torch torchvision &> /dev/null
! pip install opencv-python pycocotools matplotlib onnxruntime onnx &> /dev/null

# Download the ViT versions of the mobile SAM model weights from github repository.
!git clone https://github.com/ChaoningZhang/MobileSAM.git
%cd MobileSAM
!pip install -e .


Cloning into 'MobileSAM'...
remote: Enumerating objects: 1011, done.
remote: Counting objects: 100% (399/399), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 1011 (delta 344), reused 260 (delta 260), pack-reused 612 (from 3)
Receiving objects: 100% (1011/1011), 95.45 MiB | 22.83 MiB/s, done.
Resolving deltas: 100% (459/459), done.
/content/MobileSAM
Obtaining file:///content/MobileSAM
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mobile_sam


In [ ]:
# === CONFIGURATION ===
# Set the path to your training images and labels
image_path = "/content/drive/MyDrive/canal_project/27_11_2025/train/images"   # <-- Replace with actual image folder path
label_path = "/content/drive/MyDrive/canal_project/27_11_2025/train/masks"   # <-- Replace with actual label folder path

# === Load Image Paths ===
# Count total number of image files (e.g., .jpg format)
all_image_paths = sorted(glob(os.path.join(image_path, "*.jpg")))  # Use .png if needed
total_images = len(all_image_paths)
print(f"Total Number of Images: {total_images}")

# === Load Label Paths ===
# Count total number of label files (e.g., .png format for segmentation masks)
all_label_paths = sorted(glob(os.path.join(label_path, "*.png")))
total_labels = len(all_label_paths)
print(f"Total Number of Labels: {total_labels}")

# === Match Images and Labels ===
# Assuming both are in matching order and of equal length
train_image_paths = all_image_paths[:total_images]
train_label_paths = all_label_paths[:total_labels]

# Preview label paths (for verification)
print("Sample label paths:")
for path in train_label_paths[:5]:
    print(path)

Total Number of Images: 186
Total Number of Labels: 186
Sample label paths:
/content/drive/MyDrive/canal_project/27_11_2025/train/masks/image_capture_2025-04-05_11-45-56_jpg.rf.3d31effc5cd44f2feb5ca1c2c219346b_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/train/masks/image_capture_2025-04-06_10-10-41_jpg.rf.2d25833c6629824eb13665eb4b7a794c_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/train/masks/image_capture_2025-04-06_11-10-42_jpg.rf.ddc1a4d0ad9d75799e5114aa78022e19_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/train/masks/image_capture_2025-04-06_11-11-48_jpg.rf.6b1a6629a791c2c0c47e5ed49cca16ee_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/train/masks/image_capture_2025-04-06_12-12-56_jpg.rf.a087e1490f708cdb45e28555f3786aaf_mask.png


In [ ]:
# === CONFIGURATION ===
# Set the path to your validation images and labels
val_image_path = "/content/drive/MyDrive/canal_project/27_11_2025/valid/images"   # <-- Replace with actual validation image folder
val_label_path = "/content/drive/MyDrive/canal_project/27_11_2025/valid/masks"   # <-- Replace with actual validation label folder

# === Load Validation Image Paths ===
# Collect and sort all .jpg image files in the validation folder
val_all_image_paths = sorted(glob(os.path.join(val_image_path, "*.jpg")))
val_total_images = len(val_all_image_paths)
print(f"Total Number of Validation Images: {val_total_images}")

# === Load Validation Label Paths ===
# Collect and sort all .png label files in the validation folder
val_all_label_paths = sorted(glob(os.path.join(val_label_path, "*.png")))
val_total_labels = len(val_all_label_paths)
print(f"Total Number of Validation Labels: {val_total_labels}")

# === Match Images and Labels (by order) ===
# This assumes one-to-one correspondence between image and label files
Val1_image_paths = val_all_image_paths[:val_total_images]
Val1_label_paths = val_all_label_paths[:val_total_labels]

# Preview a few label paths to confirm loading
print("Sample validation label paths:")
for path in Val1_label_paths[:5]:
    print(path)


Total Number of Validation Images: 39
Total Number of Validation Labels: 39
Sample validation label paths:
/content/drive/MyDrive/canal_project/27_11_2025/valid/masks/image_capture_2025-04-05_11-44-49_jpg.rf.b321be12d6ab3078ae5064300cfb22ac_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/valid/masks/image_capture_2025-04-07_11-37-39_jpg.rf.2fabed3881bc90ecc487ee025e0509cb_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/valid/masks/image_capture_2025-04-08_11-04-35_jpg.rf.da8fbcd31f962ae95b3d5301f544de7e_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/valid/masks/image_capture_2025-04-09_11-31-33_jpg.rf.f831b52a22d28ffa7e8e351d3e28381b_mask.png
/content/drive/MyDrive/canal_project/27_11_2025/valid/masks/image_capture_2025-05-19_12-47-22_jpg.rf.7deac398fbf7675b3cb73df0620b94d2_mask.png


In [ ]:
# Please dont run this line if you would like to use the original size of input images.
desired_size=(640, 640)


In [ ]:
# === Load and Process Ground Truth Masks ===
# This dictionary will store binary masks where pixel > 0 is treated as True
ground_truth_masks = {}

for idx in range(len(train_label_paths)):
    # Read the label mask in grayscale
    gt_grayscale = cv2.imread(train_label_paths[idx], cv2.IMREAD_GRAYSCALE)

    # Resize the mask if desired_size is specified
    if desired_size is not None:
        gt_grayscale = cv2.resize(gt_grayscale, desired_size, interpolation=cv2.INTER_LINEAR)

    # Convert to binary mask (True where pixel > 0)
    ground_truth_masks[idx] = (gt_grayscale > 0)

# Optional: Print number of masks and preview a sample
print(f"Total ground truth masks loaded: {len(ground_truth_masks)}")
print("Example binary mask shape:", ground_truth_masks[0].shape)

Total ground truth masks loaded: 186
Example binary mask shape: (640, 640)


In [ ]:
# === Load and Process Validation Ground Truth Masks ===
# This dictionary will store binary masks for validation data
ground_truth_masksv = {}

for idx in range(len(Val1_label_paths)):
    # Read the validation label mask in grayscale
    gt_grayscale = cv2.imread(Val1_label_paths[idx], cv2.IMREAD_GRAYSCALE)

    # Resize the mask if a desired size is specified
    if desired_size is not None:
        gt_grayscale = cv2.resize(gt_grayscale, desired_size, interpolation=cv2.INTER_LINEAR)

    # Convert to binary mask: True where pixel > 0
    ground_truth_masksv[idx] = (gt_grayscale > 0)

# Print summary
print(f"Total validation ground truth masks loaded: {len(ground_truth_masksv)}")
print("Example validation mask shape:", ground_truth_masksv[0].shape)

Total validation ground truth masks loaded: 39
Example validation mask shape: (640, 640)


In [ ]:

model_type = "vit_t"
checkpoint = "/content/MobileSAM/weights/mobile_sam.pt"
device = 'cuda:0'

In [ ]:
!wget https://github.com/ChaoningZhang/MobileSAM/raw/master/weights/mobile_sam.pt


--2025-11-29 21:48:43--  https://github.com/ChaoningZhang/MobileSAM/raw/master/weights/mobile_sam.pt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt [following]
--2025-11-29 21:48:43--  https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40728226 (39M) [application/octet-stream]
Saving to: ‘mobile_sam.pt’

mobile_sam.pt       100%[===================>]  38.84M  66.6MB/s    in 0.6s    

2025-11-29 21:48:46 (66.6 MB/s) - ‘mobile_sam.pt’ saved [40728226/40728226]

In [ ]:

from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch

# === Load Pretrained SAM Model ===
sam_model = sam_model_registry[model_type](checkpoint=checkpoint)

# === Device Setup ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sam_model.to(device)

# === Set to Training Mode/Inferencing Mode (for fine-tuning) ===
sam_model.eval()
sam_model.train()

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/content/MobileSAM/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/MobileSAM/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with mobile_sam.modeling.

Sam(
  (image_encoder): TinyViT(
    (patch_embed): PatchEmbed(
      (seq): Sequential(
        (0): Conv2d_BN(
          (c): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): GELU(approximate='none')
        (2): Conv2d_BN(
          (c): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (layers): ModuleList(
      (0): ConvLayer(
        (blocks): ModuleList(
          (0-1): 2 x MBConv(
            (conv1): Conv2d_BN(
              (c): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): Conv2d_BN(
 

In [ ]:
!pip install segment-anything &> /dev/null

In [ ]:
from collections import defaultdict
from segment_anything.utils.transforms import ResizeLongestSide

# === Initialize Resize Transformation ===
# This ensures images and masks are resized while preserving aspect ratio
transform = ResizeLongestSide(sam_model.image_encoder.img_size)

# === Containers for Processed Data ===
transformed_data = defaultdict(dict)  # holds resized + preprocessed inputs for SAM
ground_truth_masks = {}               # holds binary masks for training

# === Preprocess All Training Samples ===
for k in range(len(train_image_paths)):
    # Load RGB image and grayscale ground truth mask
    image_bgr = cv2.imread(train_image_paths[k])
    mask_gray = cv2.imread(train_label_paths[k], cv2.IMREAD_GRAYSCALE)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Apply resizing transformation to both image and mask
    resized_image = transform.apply_image(image_rgb)
    resized_mask = transform.apply_image(mask_gray)

    # Convert image to torch tensor and format as [1, 3, H, W]
    image_tensor = torch.as_tensor(resized_image, device=device).permute(2, 0, 1).unsqueeze(0).contiguous()

    # Preprocess for SAM (normalization, padding, etc.)
    input_tensor = sam_model.preprocess(image_tensor)

    # Store processed image and metadata
    transformed_data[k]['image'] = input_tensor
    transformed_data[k]['input_size'] = resized_image.shape[:2][::-1]  # (W, H)
    transformed_data[k]['original_image_size'] = image_rgb.shape[:2]   # (H, W)

    # Store resized mask as binary (bool)
    ground_truth_masks[k] = (resized_mask > 0)

print(f"Preprocessed {len(transformed_data)} training samples.")

Preprocessed 186 training samples.


In [ ]:
from mobile_sam import SamPredictor

sam_model.to(device)
predictor_tuned = SamPredictor(sam_model)


In [ ]:
# === Initialize Resize Transformation ===
transform = ResizeLongestSide(sam_model.image_encoder.img_size)

# === Storage for Results ===
ground_truth_masksv = {}   # Ground truth validation masks (resized and binarized)
images_tuned_list = {}     # Resized RGB images used for prediction
masks_tuned_list = {}      # Predicted masks from SAM

# === Loop Through Validation Images ===
for s in range(len(Val1_image_paths)):
    # Load RGB image and grayscale ground truth mask
    image_bgr = cv2.imread(Val1_image_paths[s])
    gt_mask = cv2.imread(Val1_label_paths[s], cv2.IMREAD_GRAYSCALE)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Resize image and ground truth using SAM’s aspect-preserving transform
    resized_image = transform.apply_image(image_rgb)
    resized_mask = transform.apply_image(gt_mask)

    # Set resized image for SAM prediction
    predictor_tuned.set_image(resized_image)

    # Predict masks (without prompt)
    masks_tuned, _, _ = predictor_tuned.predict(
        point_coords=None,
        box=None,
        multimask_output=False
    )

    # Convert SAM prediction to binary mask
    pred_mask = masks_tuned[0]
    binary_mask = (pred_mask > 0).astype(np.float32)

    # Store outputs
    images_tuned_list[s] = resized_image
    masks_tuned_list[s] = binary_mask
    ground_truth_masksv[s] = (resized_mask > 0).astype(np.float32)

print(f"Predictions complete for {len(Val1_image_paths)} validation images.")

Predictions complete for 39 validation images.


In [ ]:
# === Training Configuration ===
keys = list(ground_truth_masks.keys())  # List of training sample indices
batch_size = 32                         # Adjust based on available GPU memory
num_epochs = 5                          # Total number of training epochs

# === Parameters to Fine-Tune ===
# You can choose to fine-tune all components or a subset
params_to_optimize = (
    list(sam_model.mask_decoder.parameters()) +        # Mask decoder (usually the main fine-tuning target)
    list(sam_model.image_encoder.parameters()) +       # Optional: include image encoder
    list(sam_model.prompt_encoder.parameters())        # Optional: include prompt encoder
)

# === Optimizer ===
# Adam optimizer with low learning rate for stable fine-tuning
optimizer = torch.optim.Adam(params_to_optimize, lr=1e-4, weight_decay=0)

# === Loss Function ===
# Binary Cross-Entropy with logits is used for binary segmentation tasks
loss_fn = torch.nn.BCEWithLogitsLoss()

In [ ]:
from torch.utils.data import DataLoader

# === Validation DataLoader Setup ===
# Here we're using a list of file paths as the dataset, which will later need to be wrapped in a proper Dataset class
val_loader = DataLoader(Val1_image_paths, batch_size=batch_size, shuffle=False)

# === Basic Validation Dataset Checks ===

# Total number of validation examples
num_val_examples = len(Val1_image_paths)
print(f"Number of validation examples: {num_val_examples}")

# Number of items returned by val_loader.dataset (same as above since it's a list)
print(f"Number of examples in validation dataset (via DataLoader): {len(val_loader.dataset)}")

# Number of batches in the validation DataLoader
print(f"Number of batches in validation loader: {len(val_loader)}")

# === Safety Check ===
# Prevent training from continuing if validation data is empty
if num_val_examples == 0:
    raise ValueError("The validation dataset is empty. Please check your data paths.")

Number of validation examples: 39
Number of examples in validation dataset (via DataLoader): 39
Number of batches in validation loader: 2


In [ ]:
torch.cuda.empty_cache()


In [ ]:
# === Accuracy Calculation Function ===
def calculate_accuracy(predictions, targets):
    """
    Computes binary accuracy between predicted and ground truth masks.
    """
    binary_predictions = (predictions > 0.5).float()
    accuracy = (binary_predictions == targets).float().mean()
    return accuracy.item()

# === Batch Training Function ===
def train_on_batch(keys, batch_start, batch_end):
    """
    Trains the SAM mask decoder on a batch of images and masks.
    Returns batch loss and accuracy.
    """
    batch_losses = []
    batch_accuracies = []

    for k in keys[batch_start:batch_end]:
        # === Get input data and metadata
        input_image = transformed_data[k]['image'].to(device)
        input_size = transformed_data[k]['input_size']
        original_image_size = transformed_data[k]['original_image_size']

        # === Forward Pass (frozen encoders)
        with torch.no_grad():
            image_embedding = sam_model.image_encoder(input_image)
            sparse_embeddings, dense_embeddings = sam_model.prompt_encoder(
                points=None, boxes=None, masks=None
            )

        low_res_masks, _ = sam_model.mask_decoder(
            image_embeddings=image_embedding,
            image_pe=sam_model.prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=False
        )

        # === Resize prediction to original size
        upscaled_masks = sam_model.postprocess_masks(
            low_res_masks, input_size, original_image_size
        ).to(device)

        # === Resize ground truth mask to match output
        gt_np = ground_truth_masks[k].astype(np.uint8)
        resized_gt = cv2.resize(gt_np, upscaled_masks.shape[-2:][::-1], interpolation=cv2.INTER_NEAREST)
        gt_binary_mask = torch.tensor(resized_gt, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)

        # === Compute loss and update weights
        loss = loss_fn(upscaled_masks, gt_binary_mask)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # === Record metrics
        batch_losses.append(loss.item())
        batch_accuracies.append(calculate_accuracy(torch.sigmoid(upscaled_masks), gt_binary_mask))

    return batch_losses, batch_accuracies

# === Epoch Training Loop ===
losses, accuracies = [], []

for epoch in range(num_epochs):
    epoch_losses = []
    epoch_accuracies = []

    print(f"\n--- EPOCH {epoch + 1}/{num_epochs} ---")

    for batch_start in range(0, len(keys), batch_size):
        batch_end = min(batch_start + batch_size, len(keys))
        batch_losses, batch_accuracies = train_on_batch(keys, batch_start, batch_end)

        batch_loss = mean(batch_losses)
        batch_accuracy = mean(batch_accuracies)
        epoch_losses.append(batch_loss)
        epoch_accuracies.extend(batch_accuracies)

        print(f'Batch [{batch_start + 1}–{batch_end}] | Loss: {batch_loss:.6f} | Accuracy: {batch_accuracy:.4f}')

    # === End of Epoch ===
    mean_train_loss = mean(epoch_losses)
    mean_train_accuracy = mean(epoch_accuracies)
    losses.append(mean_train_loss)
    accuracies.append(mean_train_accuracy)

    print(f'\nEpoch {epoch + 1} Summary:')
    print(f'➤ Mean Training Loss: {mean_train_loss:.6f}')
    print(f'➤ Mean Training Accuracy: {mean_train_accuracy:.4f}')

    # Clear cache to manage memory
    torch.cuda.empty_cache()


--- EPOCH 1/5 ---
Batch [1–32] | Loss: 0.085612 | Accuracy: 0.9641
Batch [33–64] | Loss: 0.012700 | Accuracy: 0.9957
Batch [65–96] | Loss: 0.018362 | Accuracy: 0.9929
Batch [97–128] | Loss: 0.017522 | Accuracy: 0.9936
Batch [129–160] | Loss: 0.018596 | Accuracy: 0.9939
Batch [161–186] | Loss: 0.011916 | Accuracy: 0.9953

Epoch 1 Summary:
➤ Mean Training Loss: 0.027451
➤ Mean Training Accuracy: 0.9891

--- EPOCH 2/5 ---
Batch [1–32] | Loss: 0.013629 | Accuracy: 0.9949
Batch [33–64] | Loss: 0.009086 | Accuracy: 0.9963
Batch [65–96] | Loss: 0.012932 | Accuracy: 0.9949
Batch [97–128] | Loss: 0.012691 | Accuracy: 0.9951
Batch [129–160] | Loss: 0.015278 | Accuracy: 0.9936
Batch [161–186] | Loss: 0.010367 | Accuracy: 0.9959

Epoch 2 Summary:
➤ Mean Training Loss: 0.012330
➤ Mean Training Accuracy: 0.9951

--- EPOCH 3/5 ---
Batch [1–32] | Loss: 0.011319 | Accuracy: 0.9955
Batch [33–64] | Loss: 0.008311 | Accuracy: 0.9966
Batch [65–96] | Loss: 0.012422 | Accuracy: 0.9950
Batch [97–128] | Loss:

In [ ]:

print(type(ground_truth_masksv))
print(ground_truth_masksv.keys() if isinstance(ground_truth_masksv, dict) else len(ground_truth_masksv))

<class 'dict'>
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38])


In [ ]:
from glob import glob
import os
import cv2
import numpy as np

# Set the paths to your test images and labels
test_image_dir = "/content/drive/MyDrive/canal_project/27_11_2025/test/images"
test_label_dir = "/content/drive/MyDrive/canal_project/27_11_2025/test/masks"

# === Load and Sort Test Image Paths ===
all_test_image_paths = sorted(glob(os.path.join(test_image_dir, "*.jpg")))
print(f"Total Number of Test Images: {len(all_test_image_paths)}")

# === Load and Sort Test Label Paths ===
all_test_label_paths = sorted(glob(os.path.join(test_label_dir, "*.png")))
print(f"Total Number of Test Labels: {len(all_test_label_paths)}")

# === Match Image and Label Paths ===
Test_image_paths = all_test_image_paths
Test_label_paths = all_test_label_paths

# === Count test samples (NOW IT IS SAFE) ===
test_total_images = len(Test_image_paths)
test_total_labels = len(Test_label_paths)

# === Dictionary to store binary test masks ===
ground_truth_test_masks = {}

# If you have a desired resizing size
desired_size = None   # or (512, 512)

# === Load and resize ground truth masks ===
for k in range(test_total_labels):
    gt_gray = cv2.imread(Test_label_paths[k], cv2.IMREAD_GRAYSCALE)

    if desired_size is not None:
        gt_gray = cv2.resize(gt_gray, desired_size, interpolation=cv2.INTER_NEAREST)

    ground_truth_test_masks[k] = (gt_gray > 0).astype(np.float32)

print(f"Loaded {len(ground_truth_test_masks)} ground truth test masks.")

# === Print sample paths ===
print("Sample test image path:", Test_image_paths[0] if Test_image_paths else "No images found")
print("Sample test label path:", Test_label_paths[0] if Test_label_paths else "No labels found")


Total Number of Test Images: 40
Total Number of Test Labels: 40
Loaded 40 ground truth test masks.
Sample test image path: /content/drive/MyDrive/canal_project/27_11_2025/test/images/image_capture_2025-04-06_12-11-49_jpg.rf.826be63e91371c207d97f6472d9d495b.jpg
Sample test label path: /content/drive/MyDrive/canal_project/27_11_2025/test/masks/image_capture_2025-04-06_12-11-49_jpg.rf.826be63e91371c207d97f6472d9d495b_mask.png


In [ ]:

# Dictionary to hold ground truth binary masks for test data
ground_truth_test_masks = {}

# === Load and Process Each Test Mask ===
for idx in range(len(Test_label_paths)):
    # Read label image in color (3-channel); expected mask is in the red channel
    gt_color = cv2.imread(Test_label_paths[idx])

    # Extract the red channel only and convert to binary mask
    # Note: OpenCV loads in BGR, so red is at index 2
    binary_mask = (gt_color[:, :, 2] > 0).astype(np.float32)

    # Resize if specified
    if desired_size is not None:
        binary_mask = cv2.resize(binary_mask, desired_size, interpolation=cv2.INTER_NEAREST)

    # Store in dictionary
    ground_truth_test_masks[idx] = binary_mask

print(f"Loaded {len(ground_truth_test_masks)} ground truth test masks.")

Loaded 40 ground truth test masks.


In [ ]:

# === Output containers ===
images_tuned_list = {}  # Stores resized RGB input images
masks_tuned_list = {}   # Stores binary segmentation masks

# === Run inference on all test images ===
for k in range(test_total_images):
    # Load image from disk
    image_bgr = cv2.imread(Test_image_paths[k])  # use Test_image_paths here
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Optional resizing
    if desired_size is not None:
        image_rgb = cv2.resize(image_rgb, desired_size, interpolation=cv2.INTER_LINEAR)

    # Set image for SAM predictor
    predictor_tuned.set_image(image_rgb)

    # Run prediction (no prompt input)
    masks_tuned, _, _ = predictor_tuned.predict(
        point_coords=None,
        box=None,
        multimask_output=False  # Only return most confident mask
    )

    # Convert first predicted mask to binary
    pred_mask = masks_tuned[0]  # shape: (H, W)
    binary_mask = (pred_mask > 0).astype(np.float32)

    # Store results
    images_tuned_list[k] = image_rgb
    masks_tuned_list[k] = binary_mask

print(f"Inference complete on {test_total_images} test images.")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# === Configuration ===
n_images = len(images_tuned_list)
n_cols = 4  # Images per row
n_rows = (n_images + n_cols - 1) // n_cols  # Automatically calculate needed rows

# === Create subplot grid ===
fig, axs = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))

# Ensure axs is always 2D (e.g., when n_rows = 1)
axs = np.atleast_2d(axs)

# === Loop through and display each image + overlay ===
for i in range(n_rows):
    for j in range(n_cols):
        index = i * n_cols + j
        ax = axs[i, j]

        if index < n_images:
            # Load image and binary mask
            image = images_tuned_list[index]
            mask = masks_tuned_list[index]

            # Display the RGB image
            ax.imshow(image, interpolation='none')

            # Create blue overlay for mask
            blue_overlay = np.stack([
                np.zeros_like(mask),
                np.zeros_like(mask),
                (mask > 0).astype(np.float32)
            ], axis=-1)

            # Overlay with transparency
            ax.imshow(blue_overlay, alpha=0.5)

        # Turn off axes for visual cleanliness
        ax.axis('off')

# === Adjust layout ===
plt.subplots_adjust(wspace=0.03, hspace=0.03)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import numpy as np
from sklearn.metrics import auc, roc_curve

# === Binary Metrics for One Prediction ===
def binary_segmentation_metrics(predictions, targets):
    """
    Computes binary segmentation metrics for a single predicted mask vs ground truth.
    Inputs:
        predictions (numpy array): predicted mask, float32, range [0,1] or binary
        targets (numpy array): ground truth mask, binary (0 or 1)
    Returns:
        Tuple of metrics: accuracy, precision, recall, F1-score, IoU, kappa, FP, FN, TP, TN, dice
    """
    # Flatten and convert to binary
    predictions = predictions.squeeze()
    targets = targets.squeeze()

    predictions_binary = (predictions > 0.5).astype(int)
    targets_binary = targets.astype(int)

    # Confusion matrix components
    TP = np.sum((predictions_binary == 1) & (targets_binary == 1))
    FP = np.sum((predictions_binary == 1) & (targets_binary == 0))
    FN = np.sum((predictions_binary == 0) & (targets_binary == 1))
    TN = np.sum((predictions_binary == 0) & (targets_binary == 0))

    # Metrics with small epsilon to avoid division by zero
    eps = 1e-5
    accuracy = (TP + TN + eps) / (TP + FP + FN + TN + eps)
    precision = (TP + eps) / (TP + FP + eps)
    recall = (TP + eps) / (TP + FN + eps)
    f_score = 2 * (precision * recall) / (precision + recall + eps)
    dice = (2 * TP + eps) / (2 * TP + FP + FN + eps)
    iou = (TP + eps) / (TP + FP + FN + eps)

    # Cohen’s kappa
    total = TP + FP + FN + TN
    p_o = (TP + TN) / total
    p_e = ((TP + FP) * (TP + FN) + (FN + TN) * (FP + TN)) / (total ** 2)
    kappa = (p_o - p_e) / (1 - p_e + eps)

    return accuracy, precision, recall, f_score, iou, kappa, FP, FN, TP, TN, dice

# === Average Metrics Across Dataset ===
def calculate_average_metrics(predictions_list, targets_list):
    """
    Computes average binary segmentation metrics across a dataset.
    Inputs:
        predictions_list: dictionary or list of predicted masks
        targets_list: dictionary or list of ground truth masks
    Returns:
        Dictionary of averaged metrics
    """
    num_masks = len(predictions_list)

    total_metrics = {
        'accuracy': 0.0, 'precision': 0.0, 'recall': 0.0, 'f_score': 0.0,
        'iou': 0.0, 'kappa': 0.0, 'FP': 0, 'FN': 0, 'MAR': 0.0, 'FAR': 0.0, 'dice': 0.0
    }

    for i in range(num_masks):
        pred = predictions_list[i]
        gt = targets_list[i]
        metrics = binary_segmentation_metrics(pred, gt)

        # Accumulate each metric
        for metric_name, value in zip(total_metrics.keys(), metrics):
            total_metrics[metric_name] += value

        # Add False Negative Rate (Missed Alarm Rate, MAR) and False Alarm Rate (FAR)
        TP, TN, FP, FN = metrics[8], metrics[9], metrics[6], metrics[7]
        total_metrics['MAR'] += FN / (FN + TP + 1e-5)
        total_metrics['FAR'] += FP / (FP + TN + 1e-5)

    # Compute mean for each metric
    avg_metrics = {k: v / num_masks for k, v in total_metrics.items()}

    return avg_metrics

# === Example Usage ===
# Evaluate the SAM predictions vs. ground truth test masks
avg_metrics = calculate_average_metrics(masks_tuned_list, ground_truth_test_masks)

# Print results
print("\n=== Average Metrics on Test Set ===")
for metric_name, value in avg_metrics.items():
    print(f"{metric_name.upper():<8}: {value:.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage.color import rgb2gray
import math

# === Configuration ===
border_color = 'black'
border_width = 0.35
alpha_gray = 0.5  # transparency for base grayscale image
alpha_tp = 0.5    # transparency for True Positives (blue)
alpha_fa = 0.6    # False Alarms (green)
alpha_ma = 0.6    # Missed Alarms (magenta)

# === Determine Grid Layout ===
num_images = len(images_tuned_list)
cols = 4
rows = math.ceil(num_images / cols)

# === Setup Plot ===
fig, axs = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
axs = axs.flatten()  # flatten 2D axes array for 1D indexing

# === Choose Which Images to Display (all by default) ===
selected_indices = list(range(num_images))

# === Visualization Loop ===
for i, index in enumerate(selected_indices):
    if index >= num_images:
        break

    # Convert image to grayscale
    gray_image = rgb2gray(images_tuned_list[index])

    # Get predicted and ground truth masks
    pred_mask = masks_tuned_list[index]
    gt_mask = ground_truth_test_masks[index]

    ax = axs[i]
    ax.imshow(gray_image, cmap='gray', interpolation='none', alpha=alpha_gray)

    # === Mask Overlays ===

    # True Positives: prediction and ground truth both 1 (blue)
    tp_mask = (pred_mask == 1) & (gt_mask == 1)
    tp_rgba = np.stack([np.zeros_like(tp_mask), np.zeros_like(tp_mask), tp_mask], axis=-1)
    tp_rgba = np.concatenate([tp_rgba, tp_mask[..., None].astype(float)], axis=-1)
    ax.imshow(tp_rgba, alpha=alpha_tp)

    # False Alarms: prediction is 1, ground truth is 0 (green)
    fa_mask = (pred_mask == 1) & (gt_mask == 0)
    fa_rgba = np.stack([np.zeros_like(fa_mask), fa_mask, np.zeros_like(fa_mask)], axis=-1)
    fa_rgba = np.concatenate([fa_rgba, fa_mask[..., None].astype(float)], axis=-1)
    ax.imshow(fa_rgba, alpha=alpha_fa)

    # Missed Alarms: prediction is 0, ground truth is 1 (magenta)
    ma_mask = (pred_mask == 0) & (gt_mask == 1)
    ma_rgba = np.stack([ma_mask, np.zeros_like(ma_mask), ma_mask], axis=-1)
    ma_rgba = np.concatenate([ma_rgba, ma_mask[..., None].astype(float)], axis=-1)
    ax.imshow(ma_rgba, alpha=alpha_ma)

    # Add black border
    ax.add_patch(Rectangle((0, 0), gray_image.shape[1], gray_image.shape[0],
                           linewidth=border_width, edgecolor=border_color, facecolor='none'))

    ax.axis('off')

# === Turn Off Any Unused Axes ===
for j in range(len(selected_indices), len(axs)):
    axs[j].axis('off')

# === Save and Show ===
plt.subplots_adjust(wspace=0.02, hspace=0.02)
output_path = "./ViTSAM_Evaluation_Overlay.png"  # <-- Update path as needed
plt.savefig(output_path, dpi=600, bbox_inches='tight')
plt.show()

print(f"Visualization saved to: {output_path}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage.color import rgb2gray

# === Configuration ===
border_color = 'black'
border_width = 0.35
alpha_gray = 0.5  # transparency for base grayscale image
alpha_tp = 0.5    # transparency for True Positives (blue)
alpha_fa = 0.6    # False Alarms (green)
alpha_ma = 0.6    # Missed Alarms (magenta)

# === Choose which image to display ===
index = 6  # Change this to select different image (0 to num_images-1)

# === Setup Plot ===
fig, ax = plt.subplots(figsize=(8, 8))

# Convert image to grayscale
gray_image = rgb2gray(images_tuned_list[index])

# Get predicted and ground truth masks
pred_mask = masks_tuned_list[index]
gt_mask = ground_truth_test_masks[index]

ax.imshow(gray_image, cmap='gray', interpolation='none', alpha=alpha_gray)

# === Mask Overlays ===
# True Positives: prediction and ground truth both 1 (blue)
tp_mask = (pred_mask == 1) & (gt_mask == 1)
tp_rgba = np.stack([np.zeros_like(tp_mask), np.zeros_like(tp_mask), tp_mask], axis=-1)
tp_rgba = np.concatenate([tp_rgba, tp_mask[..., None].astype(float)], axis=-1)
ax.imshow(tp_rgba, alpha=alpha_tp)

# False Alarms: prediction is 1, ground truth is 0 (green)
fa_mask = (pred_mask == 1) & (gt_mask == 0)
fa_rgba = np.stack([np.zeros_like(fa_mask), fa_mask, np.zeros_like(fa_mask)], axis=-1)
fa_rgba = np.concatenate([fa_rgba, fa_mask[..., None].astype(float)], axis=-1)
ax.imshow(fa_rgba, alpha=alpha_fa)

# Missed Alarms: prediction is 0, ground truth is 1 (magenta)
ma_mask = (pred_mask == 0) & (gt_mask == 1)
ma_rgba = np.stack([ma_mask, np.zeros_like(ma_mask), ma_mask], axis=-1)
ma_rgba = np.concatenate([ma_rgba, ma_mask[..., None].astype(float)], axis=-1)
ax.imshow(ma_rgba, alpha=alpha_ma)

# Add black border
ax.add_patch(Rectangle((0, 0), gray_image.shape[1], gray_image.shape[0],
                       linewidth=border_width, edgecolor=border_color, facecolor='none'))
ax.axis('off')

# === Save and Show ===
output_path = f"./Evaluation_Image_{index}.png"
plt.savefig(output_path, dpi=600, bbox_inches='tight')
plt.show()
print(f"Visualization saved to: {output_path}")

In [ ]:
import numpy as np

# === Metric Computation Function ===
def binary_segmentation_metrics(predictions, targets):
    predictions = predictions.squeeze()
    targets = targets.squeeze()
    predictions_binary = (predictions > 0.5).astype(int)
    targets_binary = targets.astype(int)

    TP = np.sum((predictions_binary == 1) & (targets_binary == 1))
    FP = np.sum((predictions_binary == 1) & (targets_binary == 0))
    FN = np.sum((predictions_binary == 0) & (targets_binary == 1))
    TN = np.sum((predictions_binary == 0) & (targets_binary == 0))

    eps = 1e-5
    accuracy = (TP + TN + eps) / (TP + FP + FN + TN + eps)
    precision = (TP + eps) / (TP + FP + eps)
    recall = (TP + eps) / (TP + FN + eps)
    f_score = 2 * (precision * recall) / (precision + recall + eps)
    dice = (2 * TP + eps) / (2 * TP + FP + FN + eps)
    iou = (TP + eps) / (TP + FP + FN + eps)

    total = TP + FP + FN + TN
    p_o = (TP + TN) / total
    p_e = ((TP + FP) * (TP + FN) + (FN + TN) * (FP + TN)) / (total ** 2)
    kappa = (p_o - p_e) / (1 - p_e + eps)

    return accuracy, precision, recall, f_score, iou, kappa, FP, FN, TP, TN, dice

# === Categorization Function ===
def categorize_metric(value, metric):
    categories = {
        'IoU': [(0.90, 'Excellent'), (0.85, 'Good'), (0.75, 'Fair'), (0.65, 'Poor'), (0, 'Unacceptable')],
        'Precision': [(0.95, 'Excellent'), (0.85, 'Good'), (0.65, 'Moderate'), (0, 'Fail')],
        'Kappa': [(0.88, 'Excellent'), (0.78, 'Good'), (0.68, 'Moderate'), (0, 'Fail')],
        'F-Score': [(0.88, 'Excellent'), (0.78, 'Good'), (0.68, 'Moderate'), (0, 'Fail')],
        'Recall': [(0.88, 'Excellent'), (0.78, 'Good'), (0.68, 'Moderate'), (0, 'Fail')]
    }

    if metric in categories:
        for threshold, label in categories[metric]:
            if value >= threshold:
                return label
    return "Unknown"

# === Compute Metrics for All Images ===
def compute_all_metrics(predictions_list, targets_list):
    all_metrics = []
    for i in range(len(predictions_list)):
        metrics = binary_segmentation_metrics(predictions_list[i], targets_list[i])
        metric_names = ['Accuracy', 'Precision', 'Recall', 'F-Score', 'IoU', 'Kappa', 'FP', 'FN', 'TP', 'TN', 'Dice']
        metric_dict = dict(zip(metric_names, metrics))

        # Add categorized versions
        for name in ['IoU', 'Precision', 'Kappa', 'F-Score', 'Recall']:
            metric_dict[f'{name}_Category'] = categorize_metric(metric_dict[name], name)

        all_metrics.append(metric_dict)
    return all_metrics

# === Summarize Counts and Percentages ===
def summarize_category_counts(metrics_list, metric_name, labels):
    counts = {label: 0 for label in labels}
    total = len(metrics_list)

    for m in metrics_list:
        category = m.get(f"{metric_name}_Category")
        if category in counts:
            counts[category] += 1

    # Print summary
    print(f"Counts and Percentages of Images in Each Category for {metric_name}:")
    for label in labels:
        count = counts[label]
        percentage = (count / total) * 100
        print(f"{label}: {count} ({percentage:.2f}%)")
    print()

# === Run All ===
metrics_list = compute_all_metrics(masks_tuned_list, ground_truth_test_masks)

summarize_category_counts(metrics_list, 'IoU',        ['Excellent', 'Good', 'Fair', 'Poor', 'Unacceptable'])
summarize_category_counts(metrics_list, 'Precision',  ['Excellent', 'Good', 'Moderate', 'Fail'])
summarize_category_counts(metrics_list, 'Kappa',      ['Excellent', 'Good', 'Moderate', 'Fail'])
summarize_category_counts(metrics_list, 'F-Score',    ['Excellent', 'Good', 'Moderate', 'Fail'])
summarize_category_counts(metrics_list, 'Recall',     ['Excellent', 'Good', 'Moderate', 'Fail'])


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load your image
img = cv2.imread('/content/MobileSAM/Evaluation_Image_5.png')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Get image dimensions
height, width = img.shape[:2]
print(f"Image dimensions: width={width}, height={height}")

# Display the image to see coordinates
plt.figure(figsize=(12, 8))
plt.imshow(img_rgb)
plt.title('Click to see coordinates')
plt.axis('on')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load image
img = cv2.imread('/content/MobileSAM/Evaluation_Image_6.png')

left_roi_pts = np.array([
    [550, 2500],    # top-left
    [850, 2500],    # top-right
    [900, 2900],    # bottom-right
    [600, 2900]     # bottom-left
], np.int32)

# Right ROI - same 300 pixel width as left
right_roi_pts = np.array([
    [3150, 2500],   # top-left (moved 50 left)
    [3450, 2500],   # top-right (moved 50 right)
    [3400, 2900],   # bottom-right (moved 50 right)
    [3100, 2900]    # bottom-left (moved 50 left)
], np.int32)

# Visualize both ROIs on the original image
img_with_rois = img.copy()

# Draw tilted rectangles (polygons)
cv2.polylines(img_with_rois, [left_roi_pts], True, (0, 255, 0), 2)
cv2.polylines(img_with_rois, [right_roi_pts], True, (0, 0, 255), 2)

# Add text labels
cv2.putText(img_with_rois, "LEFT", (600, 2390), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
cv2.putText(img_with_rois, "RIGHT", (3200, 2390), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# Display in Colab
cv2_imshow(img_with_rois)

# Extract ROI regions using masks
# Create mask for left ROI
mask_left = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask_left, [left_roi_pts], 255)
left_bank_roi = cv2.bitwise_and(img, img, mask=mask_left)

# Create mask for right ROI
mask_right = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask_right, [right_roi_pts], 255)
right_bank_roi = cv2.bitwise_and(img, img, mask=mask_right)

print(f"Left Bank ROI polygon: {left_roi_pts.tolist()}")
print(f"Right Bank ROI polygon: {right_roi_pts.tolist()}")

# Save
cv2.imwrite('both_rois_marked_tilted.png', img_with_rois)

# Get bounding rectangles for cropped ROIs
x_l, y_l, w_l, h_l = cv2.boundingRect(left_roi_pts)
x_r, y_r, w_r, h_r = cv2.boundingRect(right_roi_pts)

left_bank_cropped = img[y_l:y_l+h_l, x_l:x_l+w_l]
right_bank_cropped = img[y_r:y_r+h_r, x_r:x_r+w_r]

cv2.imwrite('left_bank_roi_tilted.png', left_bank_cropped)
cv2.imwrite('right_bank_roi_tilted.png', right_bank_cropped)

# Display the cropped ROIs
print("\nLeft Bank ROI (cropped):")
cv2_imshow(left_bank_cropped)

print("\nRight Bank ROI (cropped):")
cv2_imshow(right_bank_cropped)

In [ ]:
import cv2
import numpy as np

# Read the image
img = cv2.imread('both_rois_marked_tilted.png')

# Define ROIs
left_roi_pts = np.array([
    [550, 2500],
    [850, 2500],
    [900, 2900],
    [600, 2900]
], np.int32)

right_roi_pts = np.array([
    [3150, 2500],
    [3450, 2500],
    [3400, 2900],
    [3100, 2900]
], np.int32)

# Create masks for each ROI
left_mask = np.zeros(img.shape[:2], dtype=np.uint8)
right_mask = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(left_mask, [left_roi_pts], 255)
cv2.fillPoly(right_mask, [right_roi_pts], 255)

# Define blue color range (water pixels)
lower_blue = np.array([100, 0, 0])
upper_blue = np.array([255, 100, 100])

# Find blue pixels
blue_mask = cv2.inRange(img, lower_blue, upper_blue)

# Count water pixels in each ROI
left_water_pixels = cv2.countNonZero(cv2.bitwise_and(blue_mask, left_mask))
right_water_pixels = cv2.countNonZero(cv2.bitwise_and(blue_mask, right_mask))

print(f"Left ROI water pixels: {left_water_pixels}")
print(f"Right ROI water pixels: {right_water_pixels}")

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load image
img = cv2.imread('/content/MobileSAM/Evaluation_Image_5.png')

left_roi_pts = np.array([
    [550, 2500],    # top-left
    [850, 2500],    # top-right
    [900, 2900],    # bottom-right
    [600, 2900]     # bottom-left
], np.int32)

# Right ROI - same 300 pixel width as left
right_roi_pts = np.array([
    [3150, 2500],   # top-left (moved 50 left)
    [3450, 2500],   # top-right (moved 50 right)
    [3400, 2900],   # bottom-right (moved 50 right)
    [3100, 2900]    # bottom-left (moved 50 left)
], np.int32)

# Visualize both ROIs on the original image
img_with_rois = img.copy()

# Draw tilted rectangles (polygons)
cv2.polylines(img_with_rois, [left_roi_pts], True, (0, 255, 0), 2)
cv2.polylines(img_with_rois, [right_roi_pts], True, (0, 0, 255), 2)

# Add text labels
cv2.putText(img_with_rois, "LEFT", (600, 2390), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
cv2.putText(img_with_rois, "RIGHT", (3200, 2390), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# Display in Colab
cv2_imshow(img_with_rois)

# Extract ROI regions using masks
# Create mask for left ROI
mask_left = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask_left, [left_roi_pts], 255)
left_bank_roi = cv2.bitwise_and(img, img, mask=mask_left)

# Create mask for right ROI
mask_right = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask_right, [right_roi_pts], 255)
right_bank_roi = cv2.bitwise_and(img, img, mask=mask_right)

print(f"Left Bank ROI polygon: {left_roi_pts.tolist()}")
print(f"Right Bank ROI polygon: {right_roi_pts.tolist()}")

# Save
cv2.imwrite('both_rois_marked_tilted.png', img_with_rois)

# Get bounding rectangles for cropped ROIs
x_l, y_l, w_l, h_l = cv2.boundingRect(left_roi_pts)
x_r, y_r, w_r, h_r = cv2.boundingRect(right_roi_pts)

left_bank_cropped = img[y_l:y_l+h_l, x_l:x_l+w_l]
right_bank_cropped = img[y_r:y_r+h_r, x_r:x_r+w_r]

cv2.imwrite('left_bank_roi_tilted.png', left_bank_cropped)
cv2.imwrite('right_bank_roi_tilted.png', right_bank_cropped)

# Display the cropped ROIs
print("\nLeft Bank ROI (cropped):")
cv2_imshow(left_bank_cropped)

print("\nRight Bank ROI (cropped):")
cv2_imshow(right_bank_cropped)

In [ ]:
import cv2
import numpy as np

# Read the image
img = cv2.imread('both_rois_marked_tilted.png')

# Define ROIs
left_roi_pts = np.array([
    [550, 2500],
    [850, 2500],
    [900, 2900],
    [600, 2900]
], np.int32)

right_roi_pts = np.array([
    [3150, 2500],
    [3450, 2500],
    [3400, 2900],
    [3100, 2900]
], np.int32)

# Create masks for each ROI
left_mask = np.zeros(img.shape[:2], dtype=np.uint8)
right_mask = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(left_mask, [left_roi_pts], 255)
cv2.fillPoly(right_mask, [right_roi_pts], 255)

# Define blue color range (water pixels)
lower_blue = np.array([100, 0, 0])
upper_blue = np.array([255, 100, 100])

# Find blue pixels
blue_mask = cv2.inRange(img, lower_blue, upper_blue)

# Count water pixels in each ROI
left_water_pixels = cv2.countNonZero(cv2.bitwise_and(blue_mask, left_mask))
right_water_pixels = cv2.countNonZero(cv2.bitwise_and(blue_mask, right_mask))

print(f"Left ROI water pixels: {left_water_pixels}")
print(f"Right ROI water pixels: {right_water_pixels}")

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load image
img = cv2.imread('/content/MobileSAM/Evaluation_Image_10.png')

left_roi_pts = np.array([
    [550, 2500],    # top-left
    [850, 2500],    # top-right
    [900, 2900],    # bottom-right
    [600, 2900]     # bottom-left
], np.int32)

# Right ROI - same 300 pixel width as left
right_roi_pts = np.array([
    [3150, 2500],   # top-left (moved 50 left)
    [3450, 2500],   # top-right (moved 50 right)
    [3400, 2900],   # bottom-right (moved 50 right)
    [3100, 2900]    # bottom-left (moved 50 left)
], np.int32)

# Visualize both ROIs on the original image
img_with_rois = img.copy()

# Draw tilted rectangles (polygons)
cv2.polylines(img_with_rois, [left_roi_pts], True, (0, 255, 0), 2)
cv2.polylines(img_with_rois, [right_roi_pts], True, (0, 0, 255), 2)

# Add text labels
cv2.putText(img_with_rois, "LEFT", (600, 2390), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
cv2.putText(img_with_rois, "RIGHT", (3200, 2390), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# Display in Colab
cv2_imshow(img_with_rois)

# Extract ROI regions using masks
# Create mask for left ROI
mask_left = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask_left, [left_roi_pts], 255)
left_bank_roi = cv2.bitwise_and(img, img, mask=mask_left)

# Create mask for right ROI
mask_right = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask_right, [right_roi_pts], 255)
right_bank_roi = cv2.bitwise_and(img, img, mask=mask_right)

print(f"Left Bank ROI polygon: {left_roi_pts.tolist()}")
print(f"Right Bank ROI polygon: {right_roi_pts.tolist()}")

# Save
cv2.imwrite('both_rois_marked_tilted.png', img_with_rois)

# Get bounding rectangles for cropped ROIs
x_l, y_l, w_l, h_l = cv2.boundingRect(left_roi_pts)
x_r, y_r, w_r, h_r = cv2.boundingRect(right_roi_pts)

left_bank_cropped = img[y_l:y_l+h_l, x_l:x_l+w_l]
right_bank_cropped = img[y_r:y_r+h_r, x_r:x_r+w_r]

cv2.imwrite('left_bank_roi_tilted.png', left_bank_cropped)
cv2.imwrite('right_bank_roi_tilted.png', right_bank_cropped)

# Display the cropped ROIs
print("\nLeft Bank ROI (cropped):")
cv2_imshow(left_bank_cropped)

print("\nRight Bank ROI (cropped):")
cv2_imshow(right_bank_cropped)

In [ ]:
import cv2
import numpy as np

# Read the image
img = cv2.imread('both_rois_marked_tilted.png')

# Define ROIs
left_roi_pts = np.array([
    [550, 2500],
    [850, 2500],
    [900, 2900],
    [600, 2900]
], np.int32)

right_roi_pts = np.array([
    [3150, 2500],
    [3450, 2500],
    [3400, 2900],
    [3100, 2900]
], np.int32)

# Create masks for each ROI
left_mask = np.zeros(img.shape[:2], dtype=np.uint8)
right_mask = np.zeros(img.shape[:2], dtype=np.uint8)
cv2.fillPoly(left_mask, [left_roi_pts], 255)
cv2.fillPoly(right_mask, [right_roi_pts], 255)

# Define blue color range (water pixels)
lower_blue = np.array([100, 0, 0])
upper_blue = np.array([255, 100, 100])

# Find blue pixels
blue_mask = cv2.inRange(img, lower_blue, upper_blue)

# Count water pixels in each ROI
left_water_pixels = cv2.countNonZero(cv2.bitwise_and(blue_mask, left_mask))
right_water_pixels = cv2.countNonZero(cv2.bitwise_and(blue_mask, right_mask))

print(f"Left ROI water pixels: {left_water_pixels}")
print(f"Right ROI water pixels: {right_water_pixels}")